In [1]:
# autoreload magic
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from ray import tune

from mindreadingautobots.sequence_generators import make_datasets
from mindreadingautobots.models import rnn, hyperparameters  

In [22]:
# DATA LOADING
seed = 334
n_train = 800 
n_data = int(n_train * 5/4) # downstream we have a 80/20 train/val split
n_bits = 8 + 1
# k = 4
# noisy_not_majority_transition_matrix = {0: 0.05, 1: 0.05, 2: 0.05, 3: 0.95, 4: 0.95}
# noisy_not_majority_transition_matrix = {0: 0.00, 1: 0.00, 2: 0.00, 3: 1, 4: 1}
k = 2
noisy_not_majority_transition_matrix = {0: 0.00, 1: 1, 2: 1}

X, _ = make_datasets.k_lookback_weight_dataset(noisy_not_majority_transition_matrix, k, n_data, n_bits, 0, seed)
# Insert feature dimension (1 for scalar bits), and convert to tensor
stochastic_majority_data = torch.tensor(X).float().unsqueeze(-1) 
print(stochastic_majority_data.shape)
config = {
        "hidden_size": 64,
        "num_layers": 6,
        "lr": 5e-5,
        "epochs": 1000,
        "n_eval": 4,
    } 


torch.Size([1000, 9, 1])


In [9]:
(model, hidden) = rnn.train_binary_rnn(config, stochastic_majority_data, verbose=True, return_model=True)


{'hidden_size': 64, 'num_layers': 6, 'lr': 5e-05, 'epochs': 50}
{'loss': 0.6736874938011169, 'mean_accuracy': 0.625}
{'loss': 0.6440609425306321, 'mean_accuracy': 0.625}
{'loss': 0.5885090485215188, 'mean_accuracy': 0.625}
{'loss': 0.5442578300833703, 'mean_accuracy': 0.625}
{'loss': 0.49280260801315307, 'mean_accuracy': 0.625}
{'loss': 0.42149519473314284, 'mean_accuracy': 0.7727777777777778}
{'loss': 0.37829090356826783, 'mean_accuracy': 0.7744444444444445}
{'loss': 0.3873366013169289, 'mean_accuracy': 0.77}
{'loss': 0.40533560141921043, 'mean_accuracy': 0.7611111111111111}
{'loss': 0.36832432970404627, 'mean_accuracy': 0.7783333333333333}
{'loss': 0.46194894760847094, 'mean_accuracy': 0.7422222222222222}
{'loss': 0.3827341482043266, 'mean_accuracy': 0.7672222222222222}
{'loss': 0.3672057807445526, 'mean_accuracy': 0.7588888888888888}
{'loss': 0.6093201071023941, 'mean_accuracy': 0.6427777777777778}
{'loss': 0.36487396657466886, 'mean_accuracy': 0.7722222222222223}
{'loss': 0.3866143

In [12]:
X.shape

(1000, 9)

In [21]:
xx = stochastic_majority_data[:10,:-1]
yy = stochastic_majority_data[:10,1:]
print(xx.reshape(10, -1))
preds = model.predict(xx, hidden).reshape(10, -1)
print(preds)
print(yy.reshape(10, -1))

tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 1., 1., 1., 1., 1., 1.],
        [0., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 1., 1., 1., 1., 1., 1., 1.],
        [0., 1., 1., 1., 1., 1., 1., 1.],
        [1., 0., 1., 1., 1., 1., 1., 1.]])
tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 1., 1., 1., 1., 1., 1., 1.]])
tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0.,